In [1]:
import pandas as pd
import numpy as np
import os
import cv2
from glob import glob
import random
import matplotlib.pylab as plt
import gc

from keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model
from os import listdir
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score, balanced_accuracy_score
import tensorflow as tf
#import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D
%matplotlib inline

In [ ]:
# Setup inicial do notebook
#glob() Se recursive for verdadeiro, o padrão “**” corresponderá a qualquer arquivo e zero ou mais diretórios, subdiretórios e links simbólicos para diretórios.
data = glob('../input/breast-histopathology-images/IDC_regular_ps50_idx5/**/*.png', recursive=True)
len(data)
max_instances_per_class = 20000
for filename in data[1:10]:
    print(filename)

print(max_instances_per_class)

In [ ]:
# Duas matrizes contendo imagens por tipo de classe
# endswith () retorna True se uma string termina com o sufixo especificado. Caso contrário, retorna False.

classe0 = [] # 0 = benigno
classe1 = [] # 1 = maligno

for filename in data:
    if filename.endswith("class0.png"):
         classe0.append(filename)
    else:
        classe1.append(filename)
        

In [ ]:
#mostrando o tamanho de itens da classe0
len(classe0)


In [ ]:
#mostrando o tamanho de itens da classe1
len(classe1)

In [ ]:
#delimitando o tamanho de patches que serão usados
sampled_classe0 = random.sample(classe0, max_instances_per_class)
sampled_classe1 = random.sample(classe1, max_instances_per_class)
len(sampled_classe0)

In [ ]:
from matplotlib.image import imread
import cv2

#pegando os arrays das imagens

def get_image_arrays(dataa, label):
    img_arrays = []
    for i in dataa:
      if i.endswith('.png'):
        img = cv2.imread(i ,cv2.IMREAD_COLOR) #matrix da imagem indexada
        #redimensionamento e filtro gausiano
        img_sized = cv2.resize(img, (50, 50), interpolation=cv2.INTER_LINEAR) #verificar se precisa redimensionar
        img_arrays.append([img_sized, label])
    return img_arrays

In [ ]:
classe0_array = get_image_arrays(sampled_classe0, 0)
classe1_array = get_image_arrays(sampled_classe1, 1)

In [ ]:
XNC = []
yNC = []

for features,label in classe0_array:
    XNC.append(features)
    yNC.append(label)

In [ ]:
XC = []
yC = []

for features,label in classe1_array:
    XC.append(features)
    yC.append(label)

In [ ]:
# print(X[11].reshape(-1, 50, 50, 3)) placeholder
# reshape X data
XNC = np.array(XNC).reshape(-1, 50, 50, 3)
XC = np.array(XC).reshape(-1, 50, 50, 3)


In [ ]:
yNC = np.array(yNC)
yC = np.array(yC)

In [ ]:
XNC.shape

In [ ]:
yNC.shape

In [ ]:
yC.shape

In [ ]:
XC.shape

In [ ]:
#FIXANDO A CAMADA DENSA 256 - 16, 2x2
def create_classificador_1():
    classificador = Sequential()

    #add model layers
    classificador.add(Conv2D(filters=16, kernel_size=(2,2), input_shape=(50,50,3), activation='relu'))
    classificador.add(Flatten())

    classificador.add(Dense(256, activation='relu'))
    classificador.add(Dense(1, activation='sigmoid'))


    classificador.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return classificador

In [ ]:
#FIXANDO A CAMADA DENSA 256 - 32, 3x3
def create_classificador_2():
    classificador = Sequential()

    #add model layers
    classificador.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=(50,50,3), activation='relu'))
    classificador.add(Flatten())

    classificador.add(Dense(256, activation='relu'))
    classificador.add(Dense(1, activation='sigmoid'))


    classificador.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return classificador

In [ ]:
#FIXANDO A CAMADA DENSA 256 - 64, 4x4
def create_classificador_3():
    classificador = Sequential()

    #add model layers
    classificador.add(Conv2D(filters=64, kernel_size=(4,4), input_shape=(50,50,3), activation='relu'))
    classificador.add(Flatten())

    classificador.add(Dense(256, activation='relu'))
    classificador.add(Dense(1, activation='sigmoid'))


    classificador.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return classificador

In [ ]:
#FIXANDO A CAMADA DENSA 256 - C/ DUAS CONVOLUCÕES [16,32], 2x2
def create_classificador_4():
    classificador = Sequential()

    # #add model layers
    classificador.add(Conv2D(filters=16, kernel_size=(2,2), input_shape=(50,50,3), activation='relu'))
    classificador.add(Conv2D(filters=32, kernel_size=(2,2), input_shape=(50,50,3), activation='relu'))
    classificador.add(Flatten())

    classificador.add(Dense(256, activation='relu'))
    classificador.add(Dense(1, activation='sigmoid'))

    classificador.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return classificador

In [ ]:
#FIXANDO A CAMADA DENSA 256 - C/ DUAS CONVOLUCÕES [16,32], 3x3
def create_classificador_5():
    classificador = Sequential()

    # #add model layers
    classificador.add(Conv2D(filters=16, kernel_size=(3,3), input_shape=(50,50,3), activation='relu'))
    classificador.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=(50,50,3), activation='relu'))
    classificador.add(Flatten())

    classificador.add(Dense(256, activation='relu'))
    classificador.add(Dense(1, activation='sigmoid'))

    classificador.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return classificador

In [ ]:
#FIXANDO A CAMADA DENSA 256 - C/ DUAS CONVOLUCÕES [16,32], 4x4
def create_classificador_6():
    classificador = Sequential()

    # #add model layers
    classificador.add(Conv2D(filters=16, kernel_size=(4,4), input_shape=(50,50,3), activation='relu'))
    classificador.add(Conv2D(filters=32, kernel_size=(4,4), input_shape=(50,50,3), activation='relu'))
    classificador.add(Flatten())

    classificador.add(Dense(256, activation='relu'))
    classificador.add(Dense(1, activation='sigmoid'))

    classificador.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return classificador

In [ ]:
#FIXANDO A CAMADA DENSA 256 - 16, 2x2 + POLLING
def create_classificador_7():
    classificador = Sequential()

    #add model layers
    classificador.add(Conv2D(filters=16, kernel_size=(2,2), input_shape=(50,50,3), activation='relu'))
    classificador.add(MaxPool2D(pool_size = (2,2)))
    
    classificador.add(Flatten())

    classificador.add(Dense(256, activation='relu'))
    classificador.add(Dense(1, activation='sigmoid'))


    classificador.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return classificador

In [ ]:
#FIXANDO A CAMADA DENSA 256 - 32, 3x3 + POLLING
def create_classificador_8():
    classificador = Sequential()

    #add model layers
    classificador.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=(50,50,3), activation='relu'))
    classificador.add(MaxPool2D(pool_size = (2,2)))
    
    classificador.add(Flatten())

    classificador.add(Dense(256, activation='relu'))
    classificador.add(Dense(1, activation='sigmoid'))


    classificador.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return classificador

In [ ]:
#FIXANDO A CAMADA DENSA 256 - 64, 4x4 + POLLING
def create_classificador_9():
    classificador = Sequential()

    #add model layers
    classificador.add(Conv2D(filters=64, kernel_size=(4,4), input_shape=(50,50,3), activation='relu'))
    classificador.add(MaxPool2D(pool_size = (2,2)))
    
    classificador.add(Flatten())

    classificador.add(Dense(256, activation='relu'))
    classificador.add(Dense(1, activation='sigmoid'))


    classificador.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return classificador

In [ ]:
#FIXANDO A CAMADA DENSA 256 - [16,32], 2X2+ POLLING  
def create_classificador_10():
    classificador = Sequential()

    # #add model layers
    classificador.add(Conv2D(filters=16, kernel_size=(2,2), input_shape=(50,50,3), activation='relu'))
    classificador.add(MaxPooling2D(pool_size = (2,2)))
    classificador.add(Conv2D(filters=32, kernel_size=(2,2), input_shape=(50,50,3), activation='relu'))
    classificador.add(MaxPooling2D(pool_size = (2,2)))
    classificador.add(Flatten())

    classificador.add(Dense(256, activation='relu'))
    classificador.add(Dense(1, activation='sigmoid'))

    classificador.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return classificador

In [ ]:
#FIXANDO A CAMADA DENSA 256 - [16,32], 3X3+ POLLING  
def create_classificador_11():
    classificador = Sequential()

    # #add model layers
    classificador.add(Conv2D(filters=16, kernel_size=(3,3), input_shape=(50,50,3), activation='relu'))
    classificador.add(MaxPooling2D(pool_size = (2,2)))
    classificador.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=(50,50,3), activation='relu'))
    classificador.add(MaxPooling2D(pool_size = (2,2)))
    classificador.add(Flatten())

    classificador.add(Dense(256, activation='relu'))
    classificador.add(Dense(1, activation='sigmoid'))

    classificador.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return classificador

In [ ]:
#FIXANDO A CAMADA DENSA 256 - [16,32], 4X4+ POLLING  
def create_classificador_12():
    classificador = Sequential()

    # #add model layers
    classificador.add(Conv2D(filters=16, kernel_size=(4,4), input_shape=(50,50,3), activation='relu'))
    classificador.add(MaxPooling2D(pool_size = (2,2)))
    classificador.add(Conv2D(filters=32, kernel_size=(4,4), input_shape=(50,50,3), activation='relu'))
    classificador.add(MaxPooling2D(pool_size = (2,2)))
    classificador.add(Flatten())

    classificador.add(Dense(256, activation='relu'))
    classificador.add(Dense(1, activation='sigmoid'))

    classificador.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return classificador

In [ ]:
#VARIANDO A CAMADA DENSA (128) E FIXANDO PARA [16,32], 2X2 + POOLING
def create_classificador_13():
    classificador = Sequential()

    # #add model layers
    classificador.add(Conv2D(filters=16, kernel_size=(2,2), input_shape=(50,50,3), activation='relu'))
    classificador.add(MaxPooling2D(pool_size = (2,2)))
    classificador.add(Conv2D(filters=32, kernel_size=(2,2), input_shape=(50,50,3), activation='relu'))
    classificador.add(MaxPooling2D(pool_size = (2,2)))
    classificador.add(Flatten())

    classificador.add(Dense(128, activation='relu'))
    classificador.add(Dense(1, activation='sigmoid'))

    classificador.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return classificador

In [ ]:
#VARIANDO A CAMADA DENSA (256) E FIXANDO PARA [16,32], 2X2 + POOLING
def create_classificador_14():
    classificador = Sequential()

    # #add model layers
    classificador.add(Conv2D(filters=16, kernel_size=(2,2), input_shape=(50,50,3), activation='relu'))
    classificador.add(MaxPooling2D(pool_size = (2,2)))
    classificador.add(Conv2D(filters=32, kernel_size=(2,2), input_shape=(50,50,3), activation='relu'))
    classificador.add(MaxPooling2D(pool_size = (2,2)))
    classificador.add(Flatten())

    classificador.add(Dense(256, activation='relu'))
    classificador.add(Dense(1, activation='sigmoid'))

    classificador.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return classificador

In [ ]:
#VARIANDO A CAMADA DENSA (512) E FIXANDO PARA [16,32], 2X2 + POOLING
def create_classificador_15():
    classificador = Sequential()

    # #add model layers
    classificador.add(Conv2D(filters=16, kernel_size=(2,2), input_shape=(50,50,3), activation='relu'))
    classificador.add(MaxPooling2D(pool_size = (2,2)))
    classificador.add(Conv2D(filters=32, kernel_size=(2,2), input_shape=(50,50,3), activation='relu'))
    classificador.add(MaxPooling2D(pool_size = (2,2)))
    classificador.add(Flatten())

    classificador.add(Dense(512, activation='relu'))
    classificador.add(Dense(1, activation='sigmoid'))

    classificador.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return classificador

In [ ]:
#VARIANDO A CAMADA DENSA (128,128) E FIXANDO PARA [16,32], 2X2 + POOLING
def create_classificador_16():
    classificador = Sequential()

    # #add model layers
    classificador.add(Conv2D(filters=16, kernel_size=(2,2), input_shape=(50,50,3), activation='relu'))
    classificador.add(MaxPooling2D(pool_size = (2,2)))
    classificador.add(Conv2D(filters=32, kernel_size=(2,2), input_shape=(50,50,3), activation='relu'))
    classificador.add(MaxPooling2D(pool_size = (2,2)))
    classificador.add(Flatten())

    classificador.add(Dense(128, activation='relu'))
    classificador.add(Dense(128, activation='relu'))
    classificador.add(Dense(1, activation='sigmoid'))

    classificador.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return classificador

In [ ]:
#VARIANDO A CAMADA DENSA (256,256) E FIXANDO PARA [16,32], 2X2 + POOLING
def create_classificador():
    classificador = Sequential_17()

    # #add model layers
    classificador.add(Conv2D(filters=16, kernel_size=(2,2), input_shape=(50,50,3), activation='relu'))
    classificador.add(MaxPooling2D(pool_size = (2,2)))
    classificador.add(Dropout(0.3))
    classificador.add(Conv2D(filters=32, kernel_size=(2,2), input_shape=(50,50,3), activation='relu'))
    classificador.add(MaxPooling2D(pool_size = (2,2)))
    classificador.add(Flatten())

    classificador.add(Dense(256, activation='relu'))
    classificador.add(Dense(256, activation='relu'))
    classificador.add(Dropout(0.3))
    classificador.add(Dense(1, activation='sigmoid'))

    classificador.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return classificador

In [ ]:
#VARIANDO A CAMADA DENSA (512,512) E FIXANDO PARA [16,32], 2X2 + POOLING
def create_classificador_18():
    classificador = Sequential()

    # #add model layers
    classificador.add(Conv2D(filters=16, kernel_size=(2,2), input_shape=(50,50,3), activation='relu'))
    classificador.add(MaxPooling2D(pool_size = (2,2)))
    classificador.add(Conv2D(filters=32, kernel_size=(2,2), input_shape=(50,50,3), activation='relu'))
    classificador.add(MaxPooling2D(pool_size = (2,2)))
    classificador.add(Flatten())

    classificador.add(Dense(512, activation='relu'))
    classificador.add(Dense(512, activation='relu'))
    classificador.add(Dense(1, activation='sigmoid'))

    classificador.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return classificador

In [ ]:
#arquitetura ALEXNET  
#def create_classificador():
#   classificador = Sequential()

     #add model layers
#    classificador.add(Conv2D(32, kernel_size=5, activation='relu', input_shape=(50,50,3)))
#    classificador.add(MaxPooling2D(pool_size = (2,2)))
    #classificador.add(BatchNormalization())
#    classificador.add(Conv2D(32, kernel_size=5, activation='relu'))
#   classificador.add(MaxPooling2D(pool_size = (2,2)))
    #classificador.add(BatchNormalization())
#    classificador.add(Conv2D(64, kernel_size=3, activation='relu'))
#    classificador.add(Conv2D(64, kernel_size=1, activation='relu'))
#    classificador.add(Conv2D(128, kernel_size=1, activation='relu'))
#    classificador.add(MaxPooling2D(pool_size = (2,2)))
#    classificador.add(Flatten())

#   classificador.add(Dense(256, activation='relu'))
#    classificador.add(Dense(128, activation='relu'))
#    classificador.add(Dense(64, activation='relu'))
#    classificador.add(Dropout(0.5))
#    classificador.add(Dense(1, activation='softmax'))
    
#    classificador.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
#    return classificador

In [ ]:
def experimento(i):

    #separando os patches de treino e teste validação (caracterizar)
    #valor do random_state é o mesmo do valor de i
    XC_train, XC_test, yC_train, yC_test = train_test_split(XC, yC, test_size=0.25, random_state=i)
    XC_ajust, XC_val, yC_ajust, yC_val = train_test_split(XC_train, yC_train, test_size=0.10, random_state=i)
    print(XC_train.shape, XC_test.shape, yC_train.shape, yC_test.shape)
    print(XC_ajust.shape, XC_val.shape, yC_ajust.shape, yC_val.shape)
    
    XNC_train, XNC_test, yNC_train, yNC_test = train_test_split(XNC, yNC, test_size=0.25, random_state=i)
    XNC_ajust, XNC_val, yNC_ajust, yNC_val = train_test_split(XNC_train, yNC_train, test_size=0.10, random_state=i)
    print(XNC_train.shape, XNC_test.shape, yNC_train.shape, yNC_test.shape)
    print(XNC_ajust.shape, XNC_val.shape, yNC_ajust.shape, yNC_val.shape)
    
    # Merge data 
    X_train = np.concatenate((XNC_train, XC_train), axis = 0)
    y_train = np.concatenate((yNC_train, yC_train), axis = 0)


    X_test = np.concatenate((XNC_test, XC_test), axis = 0)
    y_test = np.concatenate((yNC_test, yC_test), axis = 0)


    X_ajust = np.concatenate((XNC_ajust, XC_ajust), axis = 0)
    y_ajust = np.concatenate((yNC_ajust, yC_ajust), axis = 0)


    X_val= np.concatenate((XNC_val, XC_val), axis = 0)
    y_val = np.concatenate((yNC_val, yC_val), axis = 0)

    ## dado que o range de valores possível pra um pixél vai de 0-255 
    ## escalonamos os valores entre 0-1
    ## esse processo torna nosso modelo menos variante a pequenas alterações.
    X_train= X_train / 255
    X_test = X_test / 255
    X_ajust= X_ajust / 255
    X_val = X_val / 255
    
    #repetir o experimento n vezes
    #early stopping
    #Definir a arquitetura de acordo com o número do classificador
    classificador = create_classificador_17()
    es = EarlyStopping(monitor='val_loss', mode='min', patience = 2, verbose=0)
    classificador.summary()
    plot_model(classificador, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    #history = classificador.fit(X_ajust, y_ajust, validation_data=(X_val, y_val), epochs=100, batch_size = 256, verbose = 0)
    history = classificador.fit(X_ajust, y_ajust, validation_data=(X_val, y_val), epochs=100, batch_size = 256, callbacks = [es], verbose = 0)
    loss, acc = classificador.evaluate(X_test, y_test)
    
    
    #Plots da acurácia do modelo
    plt.plot(history.history['accuracy'],label = 'treino')
    plt.plot(history.history['val_accuracy'],label = 'validação')
    plt.axhline(y=acc, color='r', linestyle='--',label = 'teste')
    plt.title('Acurácia durante as épocas')
    plt.ylabel('Acurácia')
    plt.xlabel('Épocas')
    plt.ylim((0.7,0.9))
    plt.legend(loc='upper left')
    plt.savefig('grafico acc '+str(i)+'.png', dpi=300)
    plt.show()
    
    
    #Plots do valor de perda do modelo
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Perda do modelo')
    plt.ylabel('Perda')
    plt.xlabel('Épocas')
    plt.legend(['treino', 'teste'], loc='upper left')
    plt.savefig('grafico loss '+str(i)+'.png',dpi=300)
    plt.show()
    
    #probabilidades de predição de cada classse (valores)
    y_predicted_probs = classificador.predict(X_test)
    
    #probabilidades de cada classse (labels) analisar a linha de classificação
    y_predicted = (classificador.predict(X_test) > 0.5).astype("int32").ravel()
    
    #matrix de confusão(conferir a ordem)
    tp, fp, fn, tn = confusion_matrix(y_test, y_predicted).ravel()
    
    #acuracia
    #acc = accuracy_score(y_test, y_predicted)
    #acc
    
    # calculo da acuracia ( verificacao)
    #(tp+ tn)/ (tp + tn + fp + fn) 
    
    
    
    
    #roc auc 
    auc = roc_auc_score(y_test, y_predicted_probs)
    
    #fpr ( false positive rate) ou taxa de falsos alarmes
    fpr = (fp / (fp+tn))
    
    #precisao
    prec = (tp / (tp + fp))
    
    #calculo do recall
    recall = (tp / (tp+fn))
    
    #f score
    f1 = 2 * tp / (2 * tp + fp + fn)
    
    # calculo da acuracia balanceada 
    #bac = balanced_accuracy_score(y_test, y_predicted)
    
    #especificidade
    spc = tn / (tn + fp)
    bac = (recall + spc) / 2
    
    del XC_train, XC_test, yC_train, yC_test
    del XC_ajust, XC_val, yC_ajust, yC_val
    del XNC_train, XNC_test, yNC_train, yNC_test
    del XNC_ajust, XNC_val, yNC_ajust, yNC_val
    del X_train, y_train, X_test, X_val
    del classificador
    gc.collect()
    
    return acc, bac, auc, fpr, prec, recall, f1

In [ ]:
resultados=[]
for i in range(3):
    resultados.append(experimento(i))
    

In [ ]:
import pandas as pd

df = pd.DataFrame(resultados, columns = ['ACC', 'BAC', 'AUC', 'FPR', 'PREC', 'REC', 'F1'])
df